In [1]:
import pandas as pd

In [2]:
csv_path = './PubMed.csv'
dest_path = './resultCount_cleaned.csv'

In [3]:
df = pd.read_csv(csv_path, encoding="utf-8")

In [9]:
df.head()

,TITLE,K1,K2,DOI,AUTHORS,TYPE,PUBLICATIONYEAR,URL
0,Liver manipulation causes hepatocyte injury an...,abdominal injury,gstα,10.1007/s00268-007-9182-4,"van de Poll Marcel C G,Derikx Joep P M,Buurman...",Journal Article,2007,http://sci-hub.tw/10.1007/s00268-007-9182-4
1,Intestinal-FABP and liver-FABP: Novel markers ...,abdominal injury,intestinal-fabp,10.1111/j.1553-2712.2010.00792.x,"Relja Borna,Szermutzky Maika,Henrich Dirk,Maie...",Journal Article,2010,http://sci-hub.tw/10.1111/j.1553-2712.2010.007...
2,Intestinal-FABP and liver-FABP: Novel markers ...,abdominal injury,liver-fabp,10.1111/j.1553-2712.2010.00792.x,"Relja Borna,Szermutzky Maika,Henrich Dirk,Maie...",Journal Article,2010,http://sci-hub.tw/10.1111/j.1553-2712.2010.007...
3,Paraoxonase-1 overexpression prevents experime...,abdominal injury,pon1,10.1042/CS20160185,"Burillo Elena,Tarin Carlos,Torres-Fonseca Moni...",Journal Article,2016,http://sci-hub.tw/10.1042/CS20160185
4,Paraoxonase-1 overexpression prevents experime...,abdominal injury,pon1,10.1042/CS20160185,"Burillo Elena,Tarin Carlos,Torres-Fonseca Moni...",Journal Article,2016,http://sci-hub.tw/10.1042/CS20160185


**count number of titles per combination**

*The resulting .csv needs to be cleaned to get rid of two extra columns.*

In [41]:
df[['K1', 'K2', 'TITLE']].groupby(['K1', 'K2']).count().unstack().fillna(0).to_csv(dest_path)

**get downloadable links from the urls**

In [4]:
from lxml import html
import requests
import re
from concurrent.futures import ThreadPoolExecutor

In [5]:
# save memory
rows_w_links = df[df.URL.notnull()].copy().rename(columns={'URL': 'ROOT'})
df = None

In [55]:
rows_w_links.head()

,TITLE,K1,K2,DOI,AUTHORS,TYPE,PUBLICATIONYEAR,ROOT
0,Liver manipulation causes hepatocyte injury an...,abdominal injury,gstα,10.1007/s00268-007-9182-4,"van de Poll Marcel C G,Derikx Joep P M,Buurman...",Journal Article,2007,http://sci-hub.tw/10.1007/s00268-007-9182-4
1,Intestinal-FABP and liver-FABP: Novel markers ...,abdominal injury,intestinal-fabp,10.1111/j.1553-2712.2010.00792.x,"Relja Borna,Szermutzky Maika,Henrich Dirk,Maie...",Journal Article,2010,http://sci-hub.tw/10.1111/j.1553-2712.2010.007...
2,Intestinal-FABP and liver-FABP: Novel markers ...,abdominal injury,liver-fabp,10.1111/j.1553-2712.2010.00792.x,"Relja Borna,Szermutzky Maika,Henrich Dirk,Maie...",Journal Article,2010,http://sci-hub.tw/10.1111/j.1553-2712.2010.007...
3,Paraoxonase-1 overexpression prevents experime...,abdominal injury,pon1,10.1042/CS20160185,"Burillo Elena,Tarin Carlos,Torres-Fonseca Moni...",Journal Article,2016,http://sci-hub.tw/10.1042/CS20160185
4,Paraoxonase-1 overexpression prevents experime...,abdominal injury,pon1,10.1042/CS20160185,"Burillo Elena,Tarin Carlos,Torres-Fonseca Moni...",Journal Article,2016,http://sci-hub.tw/10.1042/CS20160185


In [6]:
xpath = '/html/body/div[1]/div[1]/ul/li/a'

In [87]:
import re

# url: https://sci-hub.tw/[DOI]
def get_pdf_link(url):
    response = requests.get(url, allow_redirects=True)
    val = None
    if response: 
        tree = html.fromstring(response.content)
        string = tree.xpath('.//a[text()="⇣ save"]')
        
        if string:
            val = string[0].get('onclick')[15:-1]
            if val[:2] == '//':
                val = 'https' + val
            print('T: {} >|< {}'.format(url, val))
        else:
            val = None
            print('F: {}'.format(url))
        return val
    else:
        val = None
        print('F: no response')
        return val

# url: link from get_pdf_link
def download_from_url(url, dest):
    open(dest, 'wb').write(requests.get(url, allow_redirects=True).content)

In [88]:
rows_w_links['URL'] = rows_w_links['ROOT'].apply(get_pdf_link)

F: http://sci-hub.tw/10.1007/s00268-007-9182-4
F: http://sci-hub.tw/10.1111/j.1553-2712.2010.00792.x
F: http://sci-hub.tw/10.1111/j.1553-2712.2010.00792.x
F: http://sci-hub.tw/10.1042/CS20160185
F: http://sci-hub.tw/10.1042/CS20160185
F: http://sci-hub.tw/10.1002/ajh.23556
F: http://sci-hub.tw/10.1097/00000658-197807000-00017
F: http://sci-hub.tw/10.2478/jvetres-2018-0054
F: http://sci-hub.tw/10.1016/s0002-9610(02)01407-1
F: http://sci-hub.tw/10.1111/j.1365-2249.1993.tb03425.x
F: http://sci-hub.tw/10.2478/jvetres-2018-0054
F: http://sci-hub.tw/10.1016/s0002-9610(02)01407-1
F: http://sci-hub.tw/10.2174/1570161115666170105121900
F: http://sci-hub.tw/10.1097/TA.0b013e3181a5e7e2
F: http://sci-hub.tw/10.1111/liv.14336
F: http://sci-hub.tw/10.4103/1673-5374.194754
F: http://sci-hub.tw/10.1186/cc7923
T: http://sci-hub.tw/10.1016/j.biopha.2019.109275 >|< https//dacemirror.sci-hub.tw/journal-article/2b08209cc4861ba82a5a442e5146b1d1/zheng2019.pdf?download=true
F: http://sci-hub.tw/10.1248/bpb.b1

KeyboardInterrupt: 